## Segundo metodo Keras

##### Impotación de Librerias

In [1]:
### Ahora en Keras

from __future__ import print_function, division
from builtins import range, input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import tensorflow

from tensorflow import keras

In [2]:
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

##### Lectura del DataSet

In [3]:
df = pd.read_csv('clicks_train.csv')

In [4]:
df.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [5]:
user = df['display_id'].value_counts().index
map = {k:i for i, k in enumerate(user)}
df['display_id'] = df['display_id'].map(map)

In [6]:
mov = df['ad_id'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
df['ad_id'] = df['ad_id'].map(map)

In [7]:
df

,display_id,ad_id,clicked
0,3708577,1426,0
1,3708577,10810,0
2,3708577,82,1
3,3708577,80,0
4,3708577,22809,0
...,...,...,...
87141726,9786521,944,0
87141727,11566925,22257,1
87141728,11566925,2311,0
87141729,11566925,5476,0


##### Ordenamiento de display_id más reelevantes

Se identifican los "display_id" más reelevantes mediante una proporción "n,m" en base al tamaño total del Set de Datos "N,M". Se consideran como más reelevantes, aquellos "display_id" que mas se repiten en el dataset.

In [8]:
N = df['display_id'].max()
M = df['ad_id'].max()

N,M

(16874592, 478949)

In [9]:
n = 5
m = 5
print(n)

from collections import Counter

5


In [11]:
ucount = Counter(df['display_id'])
ucount.most_common(n)

[(8060, 12), (9291, 12), (6474, 12), (7631, 12), (11749, 12)]

In [12]:
dft = pd.DataFrame(ucount.most_common(n))
dft_list = list(dft[1])
ucount[dft_list[0]]

12

In [ ]:
mcount.most_common(m)

In [10]:
ucount = Counter(df['display_id'])
mcount = Counter(df['ad_id'])

uid = [u for u, c in ucount.most_common(n)]
mid = [u for u, c in mcount.most_common(m)]

In [11]:
mcount

Counter({1426: 11777,
         10810: 1201,
         82: 56927,
         80: 58436,
         22809: 335,
         79714: 34,
         10133: 1325,
         397756: 1,
         17112: 575,
         13310: 880,
         1013: 14975,
         4497: 3872,
         668: 20088,
         15345: 699,
         10784: 1205,
         2180: 8121,
         3425: 5219,
         344: 28835,
         2935: 6081,
         10636: 1233,
         8880: 1618,
         18063: 520,
         15351: 698,
         9386: 1481,
         43786: 96,
         38714: 120,
         45: 78505,
         639: 20906,
         35: 85552,
         1386: 11995,
         1: 203159,
         139: 45207,
         1342: 12223,
         5074: 3355,
         398940: 1,
         1546: 11081,
         4527: 3848,
         314: 30109,
         20430: 411,
         1234: 13095,
         142: 44617,
         794: 17668,
         3: 181319,
         29793: 198,
         2227: 7977,
         575: 22218,
         565: 22322,
         69: 

In [12]:
newdf = df[df['display_id'].isin(uid) & df['ad_id'].isin(mid)]
newdf.head()

,display_id,ad_id,clicked
94,1999487,5793,0
95,1999487,7038,1
147,1999493,2768,1
148,1999493,186,0
149,1999493,70,0


In [13]:
N = newdf['display_id'].max()
M = newdf['ad_id'].max()

user = newdf['display_id'].value_counts().index
map = {k:i for i, k in enumerate(user)}
newdf['display_id'] = newdf['display_id'].map(map)

C:\Users\pablo\AppData\Local\Temp\ipykernel_16488\2669345575.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['display_id'] = newdf['display_id'].map(map)


In [14]:
N,M

(2409099, 7999)

In [15]:
mov = newdf['ad_id'].value_counts().index
map = {k:i for i, k in enumerate(mov)}
newdf['ad_id'] = newdf['ad_id'].map(map)

C:\Users\pablo\AppData\Local\Temp\ipykernel_16488\1196890054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['ad_id'] = newdf['ad_id'].map(map)


##### Se Generan los Sub COnjuntos de Entrenamiento y Prueba

Una vez ordenado el DataSet, se divide en dos subconjuntos, donde el DataSet de Entrenamiento contendrá aquellos datos de "display_id" que mas se repiten, mientras que el DataSet de Prueba contendrá el 20% de los "display_id" que menos se repiten.

In [16]:
newdf = shuffle(newdf)
cutoff = int(0.8*len(newdf))
cutoff

8995870

train = newdf.iloc[: cutoff]
test = newdf.iloc[cutoff:]

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(newdf, test_size=0.2)

In [18]:
K = 10 #factores latentes
mu = newdf['clicked'].mean()
epochs = 10
reg = 0.
N = n
M = m

In [19]:
u = Input(shape=(1,))
m = Input(shape=(1,))

u_embed = Embedding(N, K, embeddings_regularizer = l2(reg))(u) # size (N,1,K)
m_embed = Embedding(M, K, embeddings_regularizer = l2(reg))(m) # size (M,1,K)

In [20]:
u_bias = Embedding(N, 1, embeddings_regularizer = l2(reg))(u)
m_bias = Embedding(M, 1, embeddings_regularizer = l2(reg))(m)

x = Dot(axes = 2)([u_embed, m_embed])

x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # N,1

In [21]:
model = Model(inputs = (u, m),
             outputs = x)

model.compile(loss = 'mse',
             optimizer = Adam(learning_rate = 0.01),
             metrics = ['mse'])

In [ ]:
r = model.fit(x = [train['display_id'].values, train['ad_id'].values],
                 y = train['clicked'].values - mu,
                 epochs = epochs,
                 batch_size = 256,
                 validation_data = ([test['display_id'].values, test['ad_id'].values],
                 test['clicked'].values - mu)
                 )

Epoch 1/10
35141/35141 [==============================] - 3709s 105ms/step - loss: 0.2024 - mse: 0.2024 - val_loss: 0.2385 - val_mse: 0.2385
Epoch 2/10
28849/35141 [=======================>......] - ETA: 10:45 - loss: 0.2466 - mse: 0.2466

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mse'], label="train mse")
plt.plot(r.history['val_mse'], label="test mse")
plt.legend()
plt.show()

## Otro 1